In [6]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.corpus import movie_reviews

In [2]:
filt = pd.read_csv("filtered data set.csv")

In [3]:
filt.head()

,Unnamed: 0,Course Key,Term Code,Comments
0,2206,200710-11026,200710,3.5 out of 5 stars
1,2207,200710-11026,200710,A lot of reading and sometimes it was boring. ...
2,2208,200710-11026,200710,"a lot of reading on your own part, but interes..."
3,2209,200710-11026,200710,"BIOS 201 is a great class,not only because of ..."
4,2210,200710-11026,200710,"Bios 201 was an organized, well taught class. ..."


In [28]:
documents = [(set(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]

In [29]:
movreviews = pd.DataFrame(documents)
movreviews.columns = ['allw','cat']

In [ ]:
allw = Counter(w.lower() for w in movie_reviews.words() if len(w) > 2)
topw = set(pd.Series(allw).sort_values(ascending = False).index[:2000])

In [41]:
movreviews['widf'] = movreviews['allw'].apply(lambda kw: {w:(True if w in kw else False) for w in topw})
forc = movreviews[['widf','cat']]
forctuple = list(forc.itertuples(index=False, name=None))

In [42]:
train_set, test_set = forctuple[100:], forctuple[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [43]:
print(nltk.classify.accuracy(classifier, test_set))

0.86


In [54]:
filt["clist"] = filt.Comments.apply(lambda x: x.split(" "))

In [56]:
filtc = filt.drop("Unnamed: 0",axis = 1)
filtc['widf'] = filtc['clist'].apply(lambda kw: {w:(True if w in kw else False) for w in topw})

In [62]:
filtc['classified'] = filtc['widf'].apply(lambda wv: classifier.classify(wv))

In [70]:
classifier.show_most_informative_features(15)

Most Informative Features
             outstanding = True              pos : neg    =     10.4 : 1.0
                  seagal = True              neg : pos    =      8.7 : 1.0
                   mulan = True              pos : neg    =      8.1 : 1.0
             wonderfully = True              pos : neg    =      6.3 : 1.0
                   damon = True              pos : neg    =      5.7 : 1.0
                    lame = True              neg : pos    =      5.6 : 1.0
                  wasted = True              neg : pos    =      5.6 : 1.0
                   awful = True              neg : pos    =      5.4 : 1.0
                   flynt = True              pos : neg    =      5.1 : 1.0
              ridiculous = True              neg : pos    =      5.1 : 1.0
                  poorly = True              neg : pos    =      4.9 : 1.0
                   waste = True              neg : pos    =      4.9 : 1.0
                    jedi = True              pos : neg    =      4.8 : 1.0

In [67]:
pd.set_option('max_colwidth', 300)
filtc[["Comments","classified"]].head(10)

,Comments,classified
0,3.5 out of 5 stars,neg
1,"A lot of reading and sometimes it was boring. Especially when there was two different people with two very different personalities where teaching it. some of the concepts were interesting, some were incredibly boring and seemed pointless. I would often say to myself that ""I dont need to know thi...",neg
2,"a lot of reading on your own part, but interesting overall",neg
3,"BIOS 201 is a great class,not only because of the material it covers, but because the class has a very relaxed atmosphere. The discussion sessions are helpful, but sometimes the preparation forms were so tedious, it didn't foster enough conversation. However it does provide a structured way for ...",neg
4,"Bios 201 was an organized, well taught class. At times a lecture method of teaching made it difficult to follow in such a large class, but mostly this was not a problem. Also, discussion sessions helped us understand certain topics in greater detail and allowed us to discuss them in a more perso...",neg
5,Dense material and too much focus on minute details. Good professors though.,neg
6,"Discussion sections good, but perhaps overused (too many weeks out of the semester). ""What's new"" a nice feature.",neg
7,Dr. Gustin is so funny. I loved his lectures.,neg
8,Extremely difficult course. I would prepare for tests by studying for at least 20 hourse in the week leading up and my graded does not \r\nreflect at all how hard I worked in this course. The hardest course next to Chemistry that I have ever taken in my life. I do not appreciate\r\nthe fact t...,neg
9,"Gustin is an amazing teacher. His lectures are insightful and interesting enough to keep me focused. He explains the topics with several different methods (such as writing on the chalkboard, handouts, verbal communication...) and this helps me understand. Gustin also has something called ""The Wh...",neg
